In [ ]:
````xml
<VSCode.Cell language="markdown">
# 🔧 Morphological Operations and Shape Analysis Pipeline
## Target: Advanced Shape Processing and Structural Analysis

This pipeline demonstrates comprehensive morphological operations:
**"Given an image with complex shapes, apply morphological operations to extract, analyze, and classify geometric structures"**

### 🎯 Problem Statement:
- Input: Binary or grayscale image with various shapes
- Goal: Use morphological operations to clean, separate, and analyze shapes
- Output: Processed shapes with geometric measurements and classifications
- Techniques: Basic morphology → Advanced operations → Shape analysis → Classification
</VSCode.Cell>

<VSCode.Cell language="python">
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage

def show_results(images, titles, rows=2, cols=3):
    """Display multiple images in a grid"""
    fig, axes = plt.subplots(rows, cols, figsize=(18, 12))
    axes = axes.flatten() if rows * cols > 1 else [axes]
    
    for i, (img, title) in enumerate(zip(images, titles)):
        if i >= len(axes):
            break
        ax = axes[i]
        if len(img.shape) == 3:
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            ax.imshow(img, cmap='gray')
        ax.set_title(title, fontweight='bold', fontsize=11)
        ax.axis('off')
    
    for j in range(i + 1, len(axes)):
        axes[j].axis('off')
    
    plt.tight_layout()
    plt.show()

def show_operation_effect(original, processed, operation_name):
    """Show before/after for morphological operations"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    ax1.imshow(original, cmap='gray')
    ax1.set_title('Before', fontweight='bold')
    ax1.axis('off')
    
    ax2.imshow(processed, cmap='gray')
    ax2.set_title('After', fontweight='bold') 
    ax2.axis('off')
    
    plt.suptitle(f'{operation_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

def analyze_shape_properties(contour):
    """Analyze geometric properties of a shape"""
    if len(contour) < 5:
        return None
    
    # Basic measurements
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    if area == 0 or perimeter == 0:
        return None
    
    # Bounding rectangle
    x, y, w, h = cv2.boundingRect(contour)
    
    # Minimum enclosing circle
    (cx, cy), radius = cv2.minEnclosingCircle(contour)
    
    # Fitted ellipse
    if len(contour) >= 5:
        ellipse = cv2.fitEllipse(contour)
        ellipse_area = np.pi * ellipse[1][0] * ellipse[1][1] / 4
    else:
        ellipse = None
        ellipse_area = 0
    
    # Shape metrics
    circularity = 4 * np.pi * area / (perimeter * perimeter)
    aspect_ratio = float(w) / h if h > 0 else 0
    extent = float(area) / (w * h) if w > 0 and h > 0 else 0
    solidity = float(area) / cv2.contourArea(cv2.convexHull(contour))
    
    # Convexity defects
    hull = cv2.convexHull(contour, returnPoints=False)
    if len(hull) > 3:
        defects = cv2.convexityDefects(contour, hull)
        convexity_defects_count = len(defects) if defects is not None else 0
    else:
        convexity_defects_count = 0
    
    return {
        'area': area,
        'perimeter': perimeter,
        'circularity': circularity,
        'aspect_ratio': aspect_ratio,
        'extent': extent,
        'solidity': solidity,
        'bounding_rect': (x, y, w, h),
        'center': (int(cx), int(cy)),
        'radius': radius,
        'ellipse': ellipse,
        'convexity_defects': convexity_defects_count
    }

def classify_shape(properties):
    """Classify shape based on geometric properties"""
    if properties is None:
        return "Unknown"
    
    circularity = properties['circularity']
    aspect_ratio = properties['aspect_ratio']
    extent = properties['extent']
    solidity = properties['solidity']
    
    # Classification rules
    if circularity > 0.85:
        return "Circle"
    elif 0.7 < aspect_ratio < 1.3 and extent > 0.8:
        return "Square"
    elif (aspect_ratio > 2.0 or aspect_ratio < 0.5) and extent > 0.7:
        return "Rectangle"
    elif 0.6 < circularity < 0.85 and len(properties.get('ellipse', [])) > 0:
        return "Ellipse"
    elif properties['convexity_defects'] > 4:
        return "Star/Complex"
    elif solidity < 0.9:
        return "Irregular"
    else:
        return "Polygon"

print("🔧 Morphological Operations and Shape Analysis Pipeline Ready!")
</VSCode.Cell>

<VSCode.Cell language="python">
# Create or load test image with various shapes
def create_test_shapes():
    """Create synthetic image with various shapes for morphological analysis"""
    img = np.zeros((500, 700, 3), dtype=np.uint8)
    
    # Add various shapes
    # Circle
    cv2.circle(img, (150, 150), 60, (255, 255, 255), -1)
    
    # Rectangle  
    cv2.rectangle(img, (300, 100), (450, 200), (255, 255, 255), -1)
    
    # Ellipse
    cv2.ellipse(img, (150, 350), (80, 40), 30, 0, 360, (255, 255, 255), -1)
    
    # Triangle
    pts = np.array([[500, 100], [600, 200], [550, 50]], np.int32)
    cv2.fillPoly(img, [pts], (255, 255, 255))
    
    # Star shape
    star_pts = []
    center = (400, 350)
    outer_radius = 50
    inner_radius = 20
    for i in range(10):
        angle = i * np.pi / 5
        radius = outer_radius if i % 2 == 0 else inner_radius
        x = int(center[0] + radius * np.cos(angle))
        y = int(center[1] + radius * np.sin(angle))
        star_pts.append([x, y])
    cv2.fillPoly(img, [np.array(star_pts, np.int32)], (255, 255, 255))
    
    # Add some noise (small objects)
    for _ in range(15):
        x, y = np.random.randint(50, 650), np.random.randint(50, 450)
        size = np.random.randint(3, 8)
        cv2.circle(img, (x, y), size, (255, 255, 255), -1)
    
    # Add some connected shapes
    cv2.rectangle(img, (500, 300), (580, 380), (255, 255, 255), -1)
    cv2.rectangle(img, (560, 320), (640, 400), (255, 255, 255), -1)
    
    # Add shapes with holes
    cv2.circle(img, (250, 250), 40, (255, 255, 255), -1)
    cv2.circle(img, (250, 250), 20, (0, 0, 0), -1)
    
    return img

# Try to load real image, fallback to synthetic
try:
    original_img = cv2.imread('D:/FPT_Material/Sem 4/CPV301/Source for PE/Image/image.jpg')
    if original_img is None:
        raise FileNotFoundError
    
    # Convert to binary
    gray = cv2.cvtColor(original_img, cv2.COLOR_BGR2GRAY)
    _, binary_img = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    test_img = cv2.cvtColor(binary_img, cv2.COLOR_GRAY2BGR)
    print("✅ Loaded and converted real image to binary")
except:
    test_img = create_test_shapes()
    print("⚠️ Created synthetic test image with various shapes")

# Convert to grayscale for morphological operations
gray_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
_, binary_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)

print(f"Image shape: {test_img.shape}")
print(f"Binary pixels: {np.sum(binary_img > 0)} foreground, {np.sum(binary_img == 0)} background")

# Show original image
plt.figure(figsize=(10, 7))
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
plt.title('Original Image with Various Shapes', fontweight='bold')
plt.axis('off')
plt.show()

plt.figure(figsize=(10, 7))
plt.imshow(binary_img, cmap='gray')
plt.title('Binary Image for Morphological Operations', fontweight='bold')
plt.axis('off')
plt.show()
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 1: Basic Morphological Operations
**Goal**: Apply fundamental morphological operations with different kernels
</VSCode.Cell>

<VSCode.Cell language="python">
# Step 1: Basic Morphological Operations
print("🔧 Step 1: Basic Morphological Operations")

# Create different kernel shapes and sizes
kernels = {
    'rect_3x3': cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3)),
    'rect_5x5': cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5)),
    'ellipse_5x5': cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)),
    'cross_5x5': cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5)),
    'custom': np.array([[0, 1, 0],
                       [1, 1, 1],
                       [0, 1, 0]], dtype=np.uint8)
}

# Basic operations with different kernels
print("Applying basic morphological operations...")

# Erosion - shrinks white regions
erosion_3x3 = cv2.erode(binary_img, kernels['rect_3x3'], iterations=1)
erosion_5x5 = cv2.erode(binary_img, kernels['rect_5x5'], iterations=1)

# Dilation - expands white regions  
dilation_3x3 = cv2.dilate(binary_img, kernels['rect_3x3'], iterations=1)
dilation_5x5 = cv2.dilate(binary_img, kernels['rect_5x5'], iterations=1)

# Opening - erosion followed by dilation (removes noise)
opening_3x3 = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernels['rect_3x3'])
opening_ellipse = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernels['ellipse_5x5'])

# Closing - dilation followed by erosion (fills holes)
closing_3x3 = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernels['rect_3x3'])
closing_5x5 = cv2.morphologyEx(binary_img, cv2.MORPH_CLOSE, kernels['rect_5x5'])

show_results(
    [binary_img, erosion_3x3, dilation_3x3, opening_3x3, closing_3x3, erosion_5x5],
    ['Original Binary', 'Erosion 3x3', 'Dilation 3x3', 'Opening 3x3', 'Closing 3x3', 'Erosion 5x5']
)

# Show effect of different kernel shapes
ellipse_opening = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernels['ellipse_5x5'])
cross_opening = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernels['cross_5x5'])
custom_opening = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernels['custom'])

show_results(
    [binary_img, opening_3x3, ellipse_opening, cross_opening, custom_opening, closing_5x5],
    ['Original', 'Rect Opening', 'Ellipse Opening', 'Cross Opening', 'Custom Opening', 'Closing 5x5']
)

print(f"✅ Basic morphological operations complete")
print(f"Pixel changes - Erosion: {np.sum(binary_img) - np.sum(erosion_3x3)} pixels removed")
print(f"Pixel changes - Dilation: {np.sum(dilation_3x3) - np.sum(binary_img)} pixels added")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 2: Advanced Morphological Operations
**Goal**: Apply advanced operations for specific shape analysis tasks
</VSCode.Cell>

<VSCode.Cell language="python">
# Step 2: Advanced Morphological Operations
print("🎯 Step 2: Advanced Morphological Operations")

# Gradient operations
# Morphological gradient - difference between dilation and erosion (edges)
gradient = cv2.morphologyEx(binary_img, cv2.MORPH_GRADIENT, kernels['rect_3x3'])

# Internal gradient - original minus erosion (inner edges)
internal_gradient = cv2.subtract(binary_img, erosion_3x3)

# External gradient - dilation minus original (outer edges)
external_gradient = cv2.subtract(dilation_3x3, binary_img)

# Top hat - original minus opening (bright spots)
tophat = cv2.morphologyEx(binary_img, cv2.MORPH_TOPHAT, kernels['rect_5x5'])

# Black hat - closing minus original (dark spots)
blackhat = cv2.morphologyEx(binary_img, cv2.MORPH_BLACKHAT, kernels['rect_5x5'])

# Hit-or-miss transform for specific pattern detection
# Define hit-miss kernel for corner detection
hit_miss_kernel = np.array([[-1, -1, -1],
                           [-1,  1, -1],
                           [ 0,  0,  0]], dtype=np.int8)

# Apply hit-or-miss (note: requires binary image with 0 and 1 values)
binary_01 = (binary_img // 255).astype(np.uint8)
hit_miss = cv2.morphologyEx(binary_01, cv2.MORPH_HITMISS, hit_miss_kernel) * 255

show_results(
    [binary_img, gradient, internal_gradient, external_gradient, tophat, blackhat],
    ['Original', 'Morphological Gradient', 'Internal Gradient', 
     'External Gradient', 'Top Hat', 'Black Hat']
)

# Advanced edge detection using morphological operations
def morphological_edge_detection(img):
    """Advanced edge detection using morphological operations"""
    # Use different sized kernels for multi-scale edge detection
    kernel_small = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    kernel_large = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    
    # Fine edges
    fine_edges = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel_small)
    
    # Coarse edges
    coarse_edges = cv2.morphologyEx(img, cv2.MORPH_GRADIENT, kernel_large)
    
    # Combined edges
    combined_edges = cv2.bitwise_or(fine_edges, coarse_edges)
    
    return fine_edges, coarse_edges, combined_edges

fine_edges, coarse_edges, combined_edges = morphological_edge_detection(binary_img)

show_results(
    [binary_img, fine_edges, coarse_edges, combined_edges, hit_miss],
    ['Original', 'Fine Edges', 'Coarse Edges', 'Combined Edges', 'Hit-or-Miss'],
    rows=2, cols=3
)

print(f"✅ Advanced morphological operations complete")
print(f"Edge pixels detected - Fine: {np.sum(fine_edges > 0)}, Coarse: {np.sum(coarse_edges > 0)}")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 3: Shape Separation and Cleanup
**Goal**: Separate connected objects and clean up the image for analysis
</VSCode.Cell>

<VSCode.Cell language="python">
# Step 3: Shape Separation and Cleanup
print("🔄 Step 3: Shape Separation and Cleanup")

def watershed_segmentation(img):
    """Apply watershed algorithm for shape separation"""
    # Ensure binary input
    if len(img.shape) == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img.copy()
    
    # Noise removal
    opening = cv2.morphologyEx(gray, cv2.MORPH_OPEN, kernels['rect_3x3'], iterations=2)
    
    # Sure background area
    sure_bg = cv2.dilate(opening, kernels['rect_3x3'], iterations=3)
    
    # Finding sure foreground area
    dist_transform = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    _, sure_fg = cv2.threshold(dist_transform, 0.7 * dist_transform.max(), 255, 0)
    
    # Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    # Marker labelling
    _, markers = cv2.connectedComponents(sure_fg)
    
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1
    
    # Mark the region of unknown with zero
    markers[unknown == 255] = 0
    
    # Apply watershed
    img_color = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
    markers = cv2.watershed(img_color, markers)
    img_color[markers == -1] = [255, 0, 0]  # Mark boundaries in red
    
    return img_color, markers, dist_transform, sure_fg, sure_bg

# Apply different cleanup techniques
# 1. Multiple opening operations to separate connected objects
separated_1 = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernels['rect_3x3'], iterations=2)
separated_2 = cv2.morphologyEx(binary_img, cv2.MORPH_OPEN, kernels['ellipse_5x5'], iterations=1)

# 2. Remove small objects
def remove_small_objects(img, min_size=100):
    """Remove objects smaller than min_size pixels"""
    # Find connected components
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=8)
    
    # Create output image
    result = np.zeros_like(img)
    
    # Keep only large components (skip background label 0)
    for i in range(1, num_labels):
        if stats[i, cv2.CC_STAT_AREA] >= min_size:
            result[labels == i] = 255
    
    return result

cleaned_large_only = remove_small_objects(binary_img, min_size=200)
cleaned_medium_only = remove_small_objects(binary_img, min_size=50)

# 3. Fill holes in objects
def fill_holes(img):
    """Fill holes in binary objects"""
    # Copy image
    result = img.copy()
    
    # Flood fill from corners to find background
    h, w = result.shape[:2]
    mask = np.zeros((h + 2, w + 2), np.uint8)
    
    # Flood fill from corners
    cv2.floodFill(result, mask, (0, 0), 255)
    
    # Invert flood filled image
    result_inv = cv2.bitwise_not(result)
    
    # Combine with original image
    filled = cv2.bitwise_or(img, result_inv)
    
    return filled

holes_filled = fill_holes(binary_img)

# 4. Watershed segmentation for better separation
watershed_result, watershed_markers, dist_transform, sure_fg, sure_bg = watershed_segmentation(binary_img)

show_results(
    [binary_img, separated_1, separated_2, cleaned_large_only, cleaned_medium_only, holes_filled],
    ['Original', 'Opening 2x', 'Ellipse Opening', 'Large Objects Only', 'Medium+ Objects', 'Holes Filled']
)

show_results(
    [binary_img, dist_transform, sure_fg, sure_bg, cv2.cvtColor(watershed_result, cv2.COLOR_BGR2RGB)],
    ['Original', 'Distance Transform', 'Sure Foreground', 'Sure Background', 'Watershed Result'],
    rows=2, cols=3
)

# Choose best cleaned image for further analysis
final_cleaned = cleaned_medium_only  # Good balance of noise removal and shape preservation

print(f"✅ Shape separation and cleanup complete")
print(f"Objects before cleanup: {np.sum(binary_img > 0)} pixels")
print(f"Objects after cleanup: {np.sum(final_cleaned > 0)} pixels")
print(f"Noise removed: {np.sum(binary_img > 0) - np.sum(final_cleaned > 0)} pixels")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 4: Shape Analysis and Measurement
**Goal**: Analyze geometric properties of detected shapes
</VSCode.Cell>

<VSCode.Cell language="python">
# Step 4: Shape Analysis and Measurement
print("📏 Step 4: Shape Analysis and Measurement")

# Find contours for shape analysis
contours, hierarchy = cv2.findContours(final_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Filter contours by area
min_area = 100
valid_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

print(f"Found {len(contours)} total contours, {len(valid_contours)} valid contours (area > {min_area})")

# Analyze each shape
shape_analysis = []
analysis_img = cv2.cvtColor(final_cleaned, cv2.COLOR_GRAY2BGR)

for i, contour in enumerate(valid_contours):
    # Get shape properties
    properties = analyze_shape_properties(contour)
    if properties is None:
        continue
    
    # Classify shape
    shape_type = classify_shape(properties)
    
    # Store analysis
    shape_analysis.append({
        'id': i + 1,
        'type': shape_type,
        'properties': properties
    })
    
    # Draw contour and annotations
    cv2.drawContours(analysis_img, [contour], -1, (0, 255, 0), 2)
    
    # Draw bounding rectangle
    x, y, w, h = properties['bounding_rect']
    cv2.rectangle(analysis_img, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    # Draw center point
    center = properties['center']
    cv2.circle(analysis_img, center, 5, (0, 0, 255), -1)
    
    # Add text label
    cv2.putText(analysis_img, f"{i+1}: {shape_type}", (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Draw minimum enclosing circle
    cv2.circle(analysis_img, center, int(properties['radius']), (255, 0, 255), 2)

# Create detailed analysis visualization
def create_shape_details(shapes):
    """Create detailed shape analysis visualization"""
    details_img = np.ones((600, 800, 3), dtype=np.uint8) * 255
    
    # Title
    cv2.putText(details_img, "SHAPE ANALYSIS RESULTS", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    
    y_pos = 70
    
    # Header
    header = "ID  Type        Area    Perim   Circ   A.Ratio  Extent  Solid"
    cv2.putText(details_img, header, (20, y_pos),
                cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
    y_pos += 25
    
    # Shape data
    for shape in shapes:
        props = shape['properties']
        line = f"{shape['id']:2d}  {shape['type']:<10} {props['area']:6.0f}  {props['perimeter']:6.1f}  {props['circularity']:5.3f}  {props['aspect_ratio']:6.2f}  {props['extent']:6.3f}  {props['solidity']:5.3f}"
        cv2.putText(details_img, line, (20, y_pos),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 0, 0), 1)
        y_pos += 20
    
    y_pos += 30
    
    # Shape classification rules
    cv2.putText(details_img, "CLASSIFICATION RULES:", (20, y_pos),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
    y_pos += 25
    
    rules = [
        "Circle:     Circularity > 0.85",
        "Square:     0.7 < Aspect Ratio < 1.3, Extent > 0.8",
        "Rectangle:  Aspect Ratio > 2.0 or < 0.5, Extent > 0.7",
        "Ellipse:    0.6 < Circularity < 0.85",
        "Star:       Convexity Defects > 4",
        "Irregular:  Solidity < 0.9",
        "Polygon:    Default classification"
    ]
    
    for rule in rules:
        cv2.putText(details_img, rule, (40, y_pos),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        y_pos += 20
    
    y_pos += 20
    
    # Measurement definitions
    cv2.putText(details_img, "MEASUREMENTS:", (20, y_pos),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
    y_pos += 25
    
    measurements = [
        "Area:       Number of pixels in shape",
        "Perimeter:  Length of shape boundary",
        "Circularity: 4π×Area/Perimeter² (1.0 = perfect circle)",
        "A.Ratio:    Width/Height of bounding rectangle",
        "Extent:     Area/Bounding Rectangle Area",
        "Solidity:   Area/Convex Hull Area"
    ]
    
    for measurement in measurements:
        cv2.putText(details_img, measurement, (40, y_pos),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        y_pos += 18
    
    return details_img

# Create visualizations
shape_details = create_shape_details(shape_analysis)

# Show results
plt.figure(figsize=(15, 8))
plt.imshow(cv2.cvtColor(analysis_img, cv2.COLOR_BGR2RGB))
plt.title('Shape Analysis with Annotations', fontweight='bold')
plt.axis('off')
plt.show()

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(shape_details, cv2.COLOR_BGR2RGB))
plt.title('Detailed Shape Analysis Results', fontweight='bold')
plt.axis('off')
plt.show()

# Print summary statistics
print(f"\n📊 Shape Analysis Summary:")
print("=" * 50)

shape_types = {}
total_area = 0
for shape in shape_analysis:
    shape_type = shape['type']
    area = shape['properties']['area']
    
    if shape_type not in shape_types:
        shape_types[shape_type] = {'count': 0, 'total_area': 0}
    
    shape_types[shape_type]['count'] += 1
    shape_types[shape_type]['total_area'] += area
    total_area += area

print(f"Total shapes analyzed: {len(shape_analysis)}")
print(f"Total area covered: {total_area:.0f} pixels")
print("\nShape distribution:")
for shape_type, data in shape_types.items():
    avg_area = data['total_area'] / data['count']
    print(f"  {shape_type}: {data['count']} shapes, avg area: {avg_area:.0f} pixels")

print(f"\n✅ Shape analysis complete")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## Step 5: Advanced Shape Processing Applications
**Goal**: Demonstrate practical applications of morphological operations
</VSCode.Cell>

<VSCode.Cell language="python">
# Step 5: Advanced Applications
print("🎉 Step 5: Advanced Shape Processing Applications")

def skeletonization(img):
    """Extract skeleton of binary shapes"""
    skeleton = np.zeros_like(img)
    element = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    
    while True:
        # Erode the image
        eroded = cv2.erode(img, element)
        # Dilate the eroded image
        temp = cv2.dilate(eroded, element)
        # Subtract the dilated image from the original
        temp = cv2.subtract(img, temp)
        # Add to skeleton
        skeleton = cv2.bitwise_or(skeleton, temp)
        img = eroded.copy()
        
        if cv2.countNonZero(img) == 0:
            break
    
    return skeleton

def shape_matching_template(img, template_shape):
    """Match shapes using template matching"""
    # Create template for specific shape (e.g., circle)
    template = np.zeros((50, 50), dtype=np.uint8)
    cv2.circle(template, (25, 25), 20, 255, -1)
    
    # Apply template matching
    result = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)
    
    # Find locations above threshold
    threshold = 0.3
    locations = np.where(result >= threshold)
    
    return result, locations, template

def convex_hull_analysis(contours):
    """Analyze convex hulls and defects"""
    hull_img = np.zeros((final_cleaned.shape[0], final_cleaned.shape[1], 3), dtype=np.uint8)
    
    for i, contour in enumerate(contours):
        if cv2.contourArea(contour) < 100:
            continue
        
        # Get convex hull
        hull = cv2.convexHull(contour)
        
        # Draw original contour in blue
        cv2.drawContours(hull_img, [contour], -1, (255, 0, 0), 2)
        
        # Draw convex hull in green
        cv2.drawContours(hull_img, [hull], -1, (0, 255, 0), 2)
        
        # Find and draw convexity defects
        hull_indices = cv2.convexHull(contour, returnPoints=False)
        if len(hull_indices) > 3:
            defects = cv2.convexityDefects(contour, hull_indices)
            if defects is not None:
                for j in range(defects.shape[0]):
                    s, e, f, d = defects[j, 0]
                    start = tuple(contour[s][0])
                    end = tuple(contour[e][0])
                    far = tuple(contour[f][0])
                    
                    # Draw defect points
                    cv2.circle(hull_img, far, 3, (0, 0, 255), -1)
                    cv2.line(hull_img, start, far, (255, 255, 0), 1)
                    cv2.line(hull_img, end, far, (255, 255, 0), 1)
    
    return hull_img

def morphological_reconstruction(img, marker):
    """Perform morphological reconstruction"""
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS, (3, 3))
    
    while True:
        # Dilate marker
        dilated = cv2.dilate(marker, kernel)
        # Intersection with mask
        reconstructed = cv2.bitwise_and(dilated, img)
        
        # Check if reconstruction is complete
        if np.array_equal(reconstructed, marker):
            break
        marker = reconstructed
    
    return reconstructed

# Apply advanced techniques
# 1. Skeletonization
print("Applying skeletonization...")
skeleton = skeletonization(final_cleaned.copy())

# 2. Shape matching
print("Performing shape matching...")
match_result, match_locations, template = shape_matching_template(final_cleaned, None)

# 3. Convex hull analysis
print("Analyzing convex hulls...")
hull_analysis = convex_hull_analysis(valid_contours)

# 4. Distance transform for shape analysis
dist_transform = cv2.distanceTransform(final_cleaned, cv2.DIST_L2, 5)
dist_transform_normalized = cv2.normalize(dist_transform, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)

# 5. Morphological reconstruction example
marker = cv2.erode(final_cleaned, kernels['rect_5x5'], iterations=2)
reconstructed = morphological_reconstruction(final_cleaned, marker)

# 6. Ultimate erosion (centers of objects)
def ultimate_erosion(img):
    """Find ultimate erosion points (object centers)"""
    dist = cv2.distanceTransform(img, cv2.DIST_L2, 5)
    _, maxima = cv2.threshold(dist, 0.7 * dist.max(), 255, cv2.THRESH_BINARY)
    return maxima.astype(np.uint8)

ultimate_erosion_result = ultimate_erosion(final_cleaned)

show_results(
    [final_cleaned, skeleton, dist_transform_normalized, 
     cv2.cvtColor(hull_analysis, cv2.COLOR_BGR2RGB), reconstructed, ultimate_erosion_result],
    ['Original Cleaned', 'Skeleton', 'Distance Transform', 
     'Convex Hull Analysis', 'Morphological Reconstruction', 'Ultimate Erosion']
)

# Create application summary
def create_application_summary():
    """Create summary of morphological applications"""
    summary_img = np.ones((700, 900, 3), dtype=np.uint8) * 255
    
    # Title
    cv2.putText(summary_img, "MORPHOLOGICAL OPERATIONS APPLICATIONS", (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
    
    y_pos = 70
    
    applications = [
        "1. NOISE REMOVAL AND CLEANUP",
        "   - Opening: Remove small noise objects",
        "   - Closing: Fill holes and gaps",
        "   - Size filtering: Remove objects by area",
        "",
        "2. SHAPE SEPARATION",
        "   - Watershed segmentation",
        "   - Multiple erosion/dilation cycles",
        "   - Distance transform analysis",
        "",
        "3. FEATURE EXTRACTION",
        "   - Skeletonization: Extract shape skeleton",
        "   - Boundary detection: Morphological gradient",
        "   - Corner detection: Hit-or-miss transform",
        "",
        "4. SHAPE ANALYSIS",
        "   - Convex hull analysis",
        "   - Convexity defects detection",
        "   - Geometric measurements",
        "",
        "5. PATTERN RECOGNITION",
        "   - Template matching",
        "   - Shape classification",
        "   - Defect detection",
        "",
        "6. PRACTICAL APPLICATIONS",
        "   - Quality control in manufacturing",
        "   - Medical image analysis",
        "   - Document processing",
        "   - Forensic analysis",
        "   - Industrial inspection",
        "   - Biological image analysis"
    ]
    
    for app in applications:
        if app.startswith(("1.", "2.", "3.", "4.", "5.", "6.")):
            cv2.putText(summary_img, app, (40, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        elif app.startswith("   -"):
            cv2.putText(summary_img, app, (60, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1)
        elif app != "":
            cv2.putText(summary_img, app, (60, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (100, 100, 100), 1)
        y_pos += 18
    
    return summary_img

application_summary = create_application_summary()

plt.figure(figsize=(12, 9))
plt.imshow(cv2.cvtColor(application_summary, cv2.COLOR_BGR2RGB))
plt.title('Morphological Operations Applications Summary', fontweight='bold')
plt.axis('off')
plt.show()

print(f"\n📊 Advanced Processing Results:")
print("-" * 40)
print(f"Skeleton pixels: {np.sum(skeleton > 0)}")
print(f"Ultimate erosion points: {np.sum(ultimate_erosion_result > 0)}")
print(f"Reconstruction difference: {np.sum(final_cleaned) - np.sum(reconstructed)} pixels")

print(f"\n💡 Practical Applications Demonstrated:")
print("1. Noise removal and image cleanup")
print("2. Shape separation and segmentation")
print("3. Skeleton extraction for shape analysis")
print("4. Convex hull analysis for defect detection")
print("5. Distance transform for object analysis")
print("6. Template matching for pattern recognition")

print(f"\n✅ Morphological Operations Pipeline Complete!")
print(f"🎯 Successfully demonstrated comprehensive shape processing")
</VSCode.Cell>

<VSCode.Cell language="markdown">
## 🎓 Exam Tips for Morphological Operations

### Operation Selection Guide:
**Problem Type → Recommended Operation**
- **Remove noise**: Opening (erosion → dilation)
- **Fill holes**: Closing (dilation → erosion)
- **Find edges**: Morphological gradient
- **Separate objects**: Watershed or multiple opening
- **Extract skeleton**: Iterative erosion with conditions
- **Detect patterns**: Hit-or-miss transform

### Kernel Selection:
**Shape and Size Guidelines:**
- **Rectangular**: General purpose, preserves straight edges
- **Elliptical**: Smoother results, preserves curves
- **Cross-shaped**: Preserves connectivity
- **Size**: Should match the feature size you want to affect

### Parameter Guidelines:
**Iterations:**
- **1-2**: Light processing, preserve details
- **3-5**: Moderate processing, good for noise removal
- **>5**: Heavy processing, major shape changes

**Kernel Sizes:**
- **3x3**: Fine details, minimal change
- **5x5**: Standard processing
- **7x7+**: Large features, significant changes

### Common Exam Scenarios:
1. **Noise removal**: Use opening with appropriate kernel
2. **Object separation**: Try watershed or iterative erosion
3. **Hole filling**: Use closing operation
4. **Edge detection**: Use morphological gradient
5. **Shape analysis**: Extract contours after cleanup

### Shape Analysis Metrics:
- **Circularity**: 4π×Area/Perimeter² (1.0 = perfect circle)
- **Aspect Ratio**: Width/Height of bounding rectangle
- **Extent**: Object area / Bounding rectangle area
- **Solidity**: Object area / Convex hull area

### Remember:
- Always choose kernel size based on feature size
- Opening removes objects smaller than kernel
- Closing fills gaps smaller than kernel
- Use appropriate connectivity (4 or 8) for your application
- Validate results visually before proceeding
- Consider computational cost for real-time applications
</VSCode.Cell>
````